**Action Recognition: Recognize human actions or activities in videos. This task requires understanding both spatial and temporal information in video data.**

La reconnaissance d'actions est un problème de vision par ordinateur et d'apprentissage automatique qui vise à identifier et classer les actions ou activités effectuées par des personnes dans des vidéos. Voici un aperçu de la façon d'aborder le problème de la reconnaissance d'actions :

    Comprendre le problème : La reconnaissance d'actions nécessite de comprendre et d'analyser des informations spatiales (apparence des objets et des personnes) et temporelles (évolution des objets et des personnes dans le temps) contenues dans les vidéos.

    Collecte des données : Rassemblez des vidéos contenant des personnes effectuant différentes actions. Vous pouvez utiliser des ensembles de données existants tels que UCF101, HMDB51 et Kinetics. Ces ensembles de données contiennent généralement des annotations fournissant les classes d'actions pour chaque vidéo.

    Prétraitement des données : Préparez les données en les normalisant, en les redimensionnant et en les échantillonnant temporellement si nécessaire. L'augmentation des données peut inclure des techniques telles que les retournements, les changements d'éclairage et les modifications de la vitesse de lecture pour améliorer la généralisation du modèle.

    Choix du modèle : Sélectionnez un modèle approprié pour la reconnaissance d'actions. Les modèles populaires pour la reconnaissance d'actions incluent les réseaux de neurones convolutifs 3D (3D CNN), les réseaux neuronaux bidirectionnels (Bi-LSTM), les réseaux de neurones convolutifs à déplacement temporel (TSN) et les réseaux Inflated 3D ConvNets (I3D).

    Entraînement du modèle : Entraînez le modèle sélectionné sur l'ensemble de données d'entraînement. Pour les modèles d'apprentissage en profondeur, vous devrez peut-être expérimenter différents hyperparamètres tels que le taux d'apprentissage, la taille du lot et le nombre d'époques d'entraînement.

    Évaluation du modèle : Évaluez les performances de votre modèle sur un ensemble de données de test distinct. Utilisez des mesures pertinentes telles que la précision, le rappel, le score F1 et la matrice de confusion pour mesurer la qualité de votre modèle.

    Amélioration et itération : En fonction des performances de votre modèle, vous devrez peut-être ajuster l'architecture du modèle, les techniques de prétraitement ou les paramètres d'entraînement. N'hésitez pas à expérimenter et à apprendre des erreurs pour améliorer continuellement votre modèle.

1. Comprendre le problème : La reconnaissance d'actions nécessite de comprendre et d'analyser des informations spatiales (apparence des objets et des personnes) et temporelles (évolution des objets et des personnes dans le temps) contenues dans les vidéos.

**Overview**

UCF101 is an action recognition data set of realistic action videos, collected from YouTube, having 101 action categories. This data set is an extension of UCF50 data set which has 50 action categories.

With 13320 videos from 101 action categories, UCF101 gives the largest diversity in terms of actions and with the presence of large variations in camera motion, object appearance and pose, object scale, viewpoint, cluttered background, illumination conditions, etc, it is the most challenging data set to date. As most of the available action recognition data sets are not realistic and are staged by actors, UCF101 aims to encourage further research into action recognition by learning and exploring new realistic action categories.
https://www.crcv.ucf.edu/research/data-sets/ucf101/<br> <br>
**Data Set Details**

The videos in 101 action categories are grouped into 25 groups, where each group can consist of 4-7 videos of an action. The videos from the same group may share some common features, such as similar background, similar viewpoint, etc.

The action categories can be divided into five types:

    Human-Object Interaction
    Body-Motion Only
    Human-Human Interaction
    Playing Musical Instruments
    Sports

The action categories for UCF101 data set are: Apply Eye Makeup, Apply Lipstick, Archery, Baby Crawling, Balance Beam, Band Marching, Baseball Pitch, Basketball Shooting, Basketball Dunk, Bench Press, Biking, Billiards Shot, Blow Dry Hair, Blowing Candles, Body Weight Squats, Bowling, Boxing Punching Bag, Boxing Speed Bag, Breaststroke, Brushing Teeth, Clean and Jerk, Cliff Diving, Cricket Bowling, Cricket Shot, Cutting In Kitchen, Diving, Drumming, Fencing, Field Hockey Penalty, Floor Gymnastics, Frisbee Catch, Front Crawl, Golf Swing, Haircut, Hammer Throw, Hammering, Handstand Pushups, Handstand Walking, Head Massage, High Jump, Horse Race, Horse Riding, Hula Hoop, Ice Dancing, Javelin Throw, Juggling Balls, Jump Rope, Jumping Jack, Kayaking, Knitting, Long Jump, Lunges, Military Parade, Mixing Batter, Mopping Floor, Nun chucks, Parallel Bars, Pizza Tossing, Playing Guitar, Playing Piano, Playing Tabla, Playing Violin, Playing Cello, Playing Daf, Playing Dhol, Playing Flute, Playing Sitar, Pole Vault, Pommel Horse, Pull Ups, Punch, Push Ups, Rafting, Rock Climbing Indoor, Rope Climbing, Rowing, Salsa Spins, Shaving Beard, Shotput, Skate Boarding, Skiing, Skijet, Sky Diving, Soccer Juggling, Soccer Penalty, Still Rings, Sumo Wrestling, Surfing, Swing, Table Tennis Shot, Tai Chi, Tennis Swing, Throw Discus, Trampoline Jumping, Typing, Uneven Bars, Volleyball Spiking, Walking with a dog, Wall Pushups, Writing On Board, Yo Yo.

2. Collecte des données : utilisation de HCF101 qui est annoté

In [ ]:
# check if HCF101 dataset is downloaded in data, if not download it


In [ ]:
import os # operating system
import glob # unix style pathname pattern expansion
from torchvision.datasets import VisionDataset # base class for vision datasets
from torchvision.datasets.folder import make_dataset # helper function to create a dataset
from torchvision.datasets.video_utils import VideoClips # helper class to handle video clips
from torchvision.transforms import ToTensor # convert a PIL Image or numpy.ndarray to tensor
import torch

class UCF101(VisionDataset):
    def __init__(self, root, frames_per_clip=16, step_between_clips=1, transform=None, target_transform=None):
        super(UCF101, self).__init__(root, transform=transform, target_transform=target_transform)

        extensions = ('avi',)
        classes = sorted(os.listdir(root))
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        self.samples = make_dataset(self.root, class_to_idx, extensions, is_valid_file=None)
        video_list = [x[0] for x in self.samples]
        self.video_clips = VideoClips(video_list, frames_per_clip, step_between_clips)

    def __len__(self):
        return self.video_clips.num_clips()

    def __getitem__(self, idx):
        video, _, _, video_idx = self.video_clips.get_clip(idx)
        target = self.samples[video_idx][1]

        if self.transform is not None:
            video = self.transform(video)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return video, target